In [14]:
import yaml

# Define the path to your YAML file
yaml_file_path = 'classes.yml'

# Open the YAML file and load its contents
with open(yaml_file_path, 'r') as file:
    config = yaml.safe_load(file)




Class: ASTGenerator
Methods:
Inputs: ['index']
Outputs: ['ast']
Class: ASTGeneratorPython
Methods:
Inputs: ['index', 'ast']
Outputs: ['ast_python']
Class: ExcelLoader
Methods:
Inputs: ['file_path']
Outputs: ['excel_file']
Class: ExcelCompatibilityChecker
Methods:
Inputs: ['excel_raw', 'excel_reduced', 'extracted_tables']
Outputs: ['is_compatible']
Class: ExcelValidator
Methods:
Inputs: ['excel_reduced']
Outputs: ['is_valid']
Class: SeriesExtractor
Methods:
Inputs: ['excel_reduced_clean', 'extracted_tables']
Outputs: ['series_data']
Class: ExcelCleaner
Methods:
Inputs: ['excel_reduced']
Outputs: ['excel_reduced_clean']
Class: TableFinder
Methods:
Inputs: ['excel_reduced_clean']
Outputs: ['extracted_tables']
Class: FormulaParser
Methods:
Inputs: ['formula']
Outputs: ['formula_ast']
Class: SeriesImplementer
Methods:
Inputs: ['formula_ast']
Outputs: ['formula_ast_series']
Class: FormulaGenerator
Methods:
Inputs: ['formula_1_ast_series', 'formula_2_ast_series']
Outputs: ['ast_generator']
Cl

In [15]:
classes[-1]

{'name': 'PythonPipelineBuilder',
 'type': 'regular',
 'description': 'Builds a Python pipeline',
 'methods': [{'name': 'build_pipeline',
   'type': 'static',
   'inputs': [{'name': 'series_data', 'type': 'dict[UUID, list[Series]]'},
    {'name': 'sorted_dag', 'type': 'list[UUID]'},
    {'name': 'ast_generator_collection',
     'type': 'dict[UUID, ASTGeneratorPython]'}],
   'outputs': [{'name': 'python_pipeline', 'type': 'str'}]}]}